In [1]:
import pandas as pd

In [2]:
content = pd.read_csv('../Data_source/sqlResult_1558435.csv',encoding='gb18030')

In [3]:
content.columns

Index(['id', 'author', 'source', 'content', 'feature', 'title', 'url'], dtype='object')

In [4]:
print(len(content))

89611


In [5]:
content.iloc[0].content

'此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作。有人猜测这也是将精力主要用到MIUI 9的研发之中。\r\nMIUI 8去年5月发布，距今已有一年有余，也是时候更新换代了。\r\n当然，关于MIUI 9的确切信息，我们还是等待官方消息。\r\n'

In [6]:
from collections import Counter

In [7]:
import jieba

In [8]:
def cut(string): 
    return list(jieba.cut(str(string)))

In [9]:
# Counter返回的是个dict，表明每个单词的个数
# Counter(cut(content.iloc[0].content))
Counter(cut(content.iloc[0].content)).values()
# sum(Counter(cut(content.iloc[0].content)).values())

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/hl/h3j1rdm12ql18b0lr03mtk580000gn/T/jieba.cache
Loading model cost 0.775 seconds.
Prefix dict has been built succesfully.


dict_values([1, 9, 1, 1, 2, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 4, 1, 1, 1, 2, 2, 1, 1, 1, 3, 3, 2, 2, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [10]:
mini_test_num = 3611

In [11]:
wordsofdocuments = [cut(content.iloc[i].content) for i in range(mini_test_num)]
words_counts_of_documents= [Counter(wordsofdocuments[i]) for i in range(mini_test_num)]
sum_of_words =[sum(words_counts_of_documents[i].values()) for i in range(mini_test_num)]

In [12]:
# 定义word在第i个document中的频率： (word在第i个document出现的次数)/(第i个document中word的个数)
def term_frequency(word, i):
    # words = cut(content.iloc[i].content)
    # words = wordsofdocuments[i] 
    # words_counts= Counter(words)
    words_counts = words_counts_of_documents[i]
    if (words_counts[word]==0): return 0
    #return words_counts[word]/sum(words_counts.values())
    return words_counts[word]/sum_of_words[i]

In [13]:
term_frequency('小米', 0)

0.009615384615384616

In [14]:
# documents_words_counter =[
#     Counter(cut(str(content.iloc[i].content))) for i in range(mini_test_num)
# ]
documents_words_counter=words_counts_of_documents

In [19]:
import numpy as np
from functools import reduce
import operator as op

In [20]:
all_words_counter = reduce(op.add, documents_words_counter)
len(all_words_counter)
# 所有word的个数

90352

In [23]:
all_words_counter['小米']
sum_word_in_document = reduce(op.add, documents_words_counter)
for word in sum_word_in_document:
    sum_word_of_doucments=sum(1 for counter in documents_words_counter if word in counter)
    sum_word_in_document[word]=sum_word_of_doucments

        

In [24]:
# word的反向文件频率inverse document frequency:  log(文件样本数目/有word出现的文件样本数目)
length=len(documents_words_counter)
def inverse_document_frequency(word):
    # return np.log(len(documents_words_counter)/sum(1 for counter in documents_words_counter if word in counter))
    return np.log(length/sum_word_in_document[word])

In [25]:
inverse_document_frequency('小米')

4.933643483255976

In [26]:
# all_words_counter
# sum_word_in_document

In [27]:
# tfidf:word在第i个文档tdidf：term_frequency越高，word在该文档中越有分量， 但是如果word是个通用的stopword，idf会越小，word分量就减轻了
def tfidf(word, i): 
    tf=term_frequency(word, i)
    if(tf==0): return 0
    return tf * inverse_document_frequency(word)

In [28]:
tfidf('小米', 0)

0.04743887964669208

In [29]:
documents_words_counter[0].most_common(15)

[('，', 9),
 ('。', 4),
 ('MIUI', 3),
 (' ', 3),
 ('\r\n', 3),
 ('（', 2),
 ('6', 2),
 ('月', 2),
 ('）', 2),
 ('机型', 2),
 ('发布', 2),
 ('精力', 2),
 ('也', 2),
 ('是', 2),
 ('9', 2)]

In [30]:
tfidf_of_document_0 ={
    word : tfidf(word, 0) for word in documents_words_counter[0]
}

In [31]:
# tfidf_of_document_0.items()

In [32]:
sorted(tfidf_of_document_0.items(), key= lambda x:x[1], reverse= True)[:15]

[('MIUI', 0.23630019292146512),
 ('机型', 0.10194938968040948),
 ('精力', 0.10090963542521188),
 ('体验版', 0.07876673097382171),
 ('稳定版', 0.07876673097382171),
 ('暂不受', 0.07876673097382171),
 ('系统优化', 0.07876673097382171),
 ('距今已有', 0.07876673097382171),
 ('内测', 0.07210185423766839),
 ('用到', 0.06543697750151507),
 ('有余', 0.06329136643118613),
 ('更新换代', 0.06329136643118613),
 ('官方消息', 0.06005605646367447),
 ('确切', 0.05571004565845276),
 ('小米', 0.04743887964669208)]

In [33]:
from functools import reduce

In [34]:
import operator as op

In [35]:
# 第一个document的向量， 向量中的每个元素对于于word的tfidf
document_0 = [tfidf(w, 0) for w in all_words_counter]

In [36]:
## tfidf ==> sci-kit learning; 

In [37]:
# 项目开始

In [38]:
sources = [content.iloc[i].source for i in range(mini_test_num)]

In [39]:
# documents = [[tfidf(w, i) for w in all_words_counter] for i in range(mini_test_num)]
def documentize(i):
    return [tfidf(w, i) for w in all_words_counter]


In [40]:
# documents=list(map(documentize, range(1)))

In [41]:
# document_x =documentize(0)

In [42]:
documents=list(map(documentize, range(mini_test_num)))

In [43]:
def naming(s):
    i = s.find('@')
    if (i==-1): return s
    else: return s[:i]
sources_consice = list(map(naming, sources))
# naming('新华社@http://')

In [44]:
# for i in range(len(sources_consice)):
#     if(sources_consice[i]=='新华社'):
#         print("新华社")

In [45]:
y_name, y = np.unique(['abc','a','abc','b','c'], return_inverse=True)
print(y_name)
print(y)
print(y_name[y])


['a' 'abc' 'b' 'c']
[1 0 1 2 3]
['abc' 'a' 'abc' 'b' 'c']


In [46]:

y_name, y = np.unique(sources_consice, return_inverse=True)

In [47]:
index = -1
print(sources_consice[3])
print(y[3])
print("---")
for i in range(len(sources_consice)):
    if(sources_consice[i]=='新华社'):
        index=y[i]
        print(i)
        print(y[i])
        print(y_name[y[i]])
print(len(y_name))
print(len(y))


新华社
264
---
3
264
新华社
39
264
新华社
65
264
新华社
95
264
新华社
113
264
新华社
305
264
新华社
312
264
新华社
314
264
新华社
543
264
新华社
545
264
新华社
651
264
新华社
671
264
新华社
679
264
新华社
715
264
新华社
731
264
新华社
738
264
新华社
794
264
新华社
810
264
新华社
816
264
新华社
909
264
新华社
931
264
新华社
934
264
新华社
937
264
新华社
940
264
新华社
943
264
新华社
946
264
新华社
951
264
新华社
954
264
新华社
958
264
新华社
964
264
新华社
966
264
新华社
990
264
新华社
995
264
新华社
1005
264
新华社
1009
264
新华社
1011
264
新华社
1015
264
新华社
1066
264
新华社
1113
264
新华社
1141
264
新华社
1143
264
新华社
1146
264
新华社
1150
264
新华社
1152
264
新华社
1155
264
新华社
1180
264
新华社
1182
264
新华社
1185
264
新华社
1188
264
新华社
1204
264
新华社
1520
264
新华社
1666
264
新华社
1670
264
新华社
1688
264
新华社
1691
264
新华社
1719
264
新华社
1763
264
新华社
1766
264
新华社
1999
264
新华社
2250
264
新华社
2252
264
新华社
2473
264
新华社
2629
264
新华社
2760
264
新华社
2781
264
新华社
2803
264
新华社
2805
264
新华社
2809
264
新华社
2813
264
新华社
2830
264
新华社
2833
264
新华社
3030
264
新华社
3056
264
新华社
3082
264
新华社
3093
264
新华社
3142
264
新华社
3189
264
新华社
3206
264
新华社
3226
264
新华社

In [49]:
len(documents)

3611

In [50]:
n_neighbors = 10
def _weight_func(i):
    i=1/i
    i[:,0]=i[:,0]-i[:,0]
    return i
from sklearn import neighbors
clf = neighbors.KNeighborsClassifier(n_neighbors, weights=_weight_func)
clf.fit(documents, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights=<function _weight_func at 0x11a7c2f28>)

In [53]:
to_be_predict =[documents[i] ]
result = []
for i in range(len(documents)):
    to_be_predict= [documents[i]]
    z=clf.predict(to_be_predict)
    if(z[0]==index):
        print(i) 
        result.append(i)


/anaconda2/envs/nlppy3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda2/envs/nlppy3/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in subtract
  after removing the cwd from sys.path.


127
570
581
599
618
903
982
2238
2299
2301
2737
2775
3066
3076
3133
3211
3214
3273


In [85]:
array = [127,
570,
581,
599,
618,
903,
982,
2238,
2299,
2301,
2737,
2775,
3066,
3076,
3133,
3211,
3214,
3273]

for i in result:
    print(sources_consice[i])
for i in array:
    print(sources_consice[i])


经济日报
新华网
新华网
新华网
新华网
新华网
新华网
新华网
新华网
新华网
新华网
新华网
新华网
澎湃新闻
证券日报
新华网
环球时报
经济参考报


In [54]:
y_name[264]

'新华社'